## Indirect Object Identification, part 1: The Tokenizers
This notebook is part of an attempt to reproduce the experiments in the paper ["Interpretability in The Wild"](https://arxiv.org/abs/2211.00593). This paper was focused on the indirect object identification (IOI) task, which involves observing a language model's prediction of the last word of a sentence that contains an indirect object (IO) and a subject (S). If the observation is the Indirect Object, We initially generated a dataset of 100000 sentences using names, objects and places generated by GPT-3 (text-davinci-002). This process generated both single-token and multi-token words, which will make this a little more difficult. There are ways to mitigate this, including using each model's tokenizer to generate tokens, filtering the single-token items into the set that will end up in the study. However, this has a pitfall in that each tokenizer has a different behaviour! Here are my observations on this phenomenon.

Observations:
+ 80 / 504 of the names are single-token names according to gpt2-small
+ 0 / 504 of the names are tokenized as single token names according to opt-125m
+ 58 / 504 of the names are tokenized as single tokens according to solu-8l-old


I estimate that there is an even wider variance of this particular observation across all implementations of positional encoding, maybe characterizing this with a sweep would be informative.

In [21]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import einops
from fancy_einsum import einsum
from dataclasses import dataclass
from names import PERSON
from easy_transformer import EasyTransformer

In [11]:
model = EasyTransformer.from_pretrained("opt-125m")

Loading model: opt-125m


Downloading:   0%|          | 0.00/651 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/685 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/441 [00:00<?, ?B/s]

Moving model to device:  cuda
Finished loading pretrained model opt-125m into EasyTransformer!


In [17]:
model_solu = EasyTransformer.from_pretrained("solu-8l-old")

Loading model: solu-8l-old


Using pad_token, but it is not set yet.


Moving model to device:  cuda
Finished loading pretrained model solu-8l-old into EasyTransformer!


In [14]:
# people = []
people_opt = []
multi_tok = []
for person in PERSON:
    tok = model.to_str_tokens(person, prepend_bos=False)
    if len(tok) == 1:
        people_opt.append(person)
    else:
        multi_tok.append(tok)

In [18]:
# people = []
# people_opt = []
people_solu = []
#multi_tok = []
multi_tok_solu = []
for person in PERSON:
    tok = model_solu.to_str_tokens(person, prepend_bos=False)
    if len(tok) == 1:
        people_solu.append(person)
    else:
        multi_tok_solu.append(tok)

In [10]:
len(people)

80

In [13]:
len(people_opt)

0

In [15]:
len(multi_tok)

504

In [23]:
model_solu.cfg

EasyTransformerConfig:
{'act_fn': 'solu_ln',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 64,
 'd_mlp': 4096,
 'd_model': 1024,
 'd_vocab': 50278,
 'd_vocab_out': 50278,
 'device': 'cuda',
 'eps': 1e-05,
 'final_rms': False,
 'from_checkpoint': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.025,
 'model_name': 'SoLU_8L_v21_old',
 'n_ctx': 1024,
 'n_heads': 16,
 'n_layers': 8,
 'n_params': 100663296,
 'normalization_type': 'LNPre',
 'original_architecture': 'neel-solu-old',
 'parallel_attn_mlp': False,
 'positional_embedding_type': 'standard',
 'rotary_dim': None,
 'scale_attn_by_inverse_layer_idx': False,
 'seed': 42,
 'tokenizer_name': 'EleutherAI/gpt-neox-20b',
 'use_attn_result': False,
 'use_attn_scale': True,
 'use_local_attn': False,
 'window_size': None}

In [16]:
model.cfg

EasyTransformerConfig:
{'act_fn': 'relu',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 64,
 'd_mlp': 3072,
 'd_model': 768,
 'd_vocab': 50272,
 'd_vocab_out': 50272,
 'device': 'cuda',
 'eps': 1e-05,
 'final_rms': False,
 'from_checkpoint': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.02886751345948129,
 'model_name': 'opt-125m',
 'n_ctx': 2048,
 'n_heads': 12,
 'n_layers': 12,
 'n_params': 84934656,
 'normalization_type': 'LNPre',
 'original_architecture': 'OPTForCausalLM',
 'parallel_attn_mlp': False,
 'positional_embedding_type': 'standard',
 'rotary_dim': None,
 'scale_attn_by_inverse_layer_idx': False,
 'seed': 42,
 'tokenizer_name': 'facebook/opt-125m',
 'use_attn_result': False,
 'use_attn_scale': True,
 'use_local_attn': False,
 'window_size': None}

In [19]:
len(people_solu)

58

In [20]:
len(multi_tok_solu)

446